In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from sklearn import preprocessing

In [2]:
data_a_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_A.csv", low_memory = False);
data_b_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_B.csv", low_memory = False);
data_c_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_C.csv", low_memory = False);
data_d_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_D.csv", low_memory = False);

data_a_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_A.csv", low_memory = False);
data_b_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_B.csv", low_memory = False);
data_c_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_C.csv", low_memory = False);
data_d_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_D.csv", low_memory = False);
data_e_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_E.csv", low_memory = False);
data_f_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_F.csv", low_memory = False);
data_g_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_G.csv", low_memory = False);
data_h_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_H.csv", low_memory = False);

In [74]:
data_2014 = pd.concat([data_a_2014, data_b_2014, data_c_2014, data_d_2014], ignore_index=True)

data_2015 = pd.concat([data_a_2015, data_b_2015, data_c_2015, data_d_2015, 
                       data_e_2015, data_f_2015, data_g_2015, data_h_2015], 
                       ignore_index=True)

In [75]:
def format_revol(val):
    percent = val[:-1]
    return float(percent)/100

def format_interest(val):
    percent = val[:-1]
    return float(percent)/100

def format_term(val):
    term = val[1:3]
    return float(term)/12

In [110]:
def initial_model(df, columns, columns_to_encode, columns_to_normalize):
    min_max_scaler = preprocessing.MinMaxScaler()
    new_df = df.copy()
    #print(new_df.zip_code.unique())
    new_df = new_df[columns]
    new_df = pd.get_dummies(new_df, columns = columns_to_encode)
    new_df['amnt'] = new_df['funded_amnt']
    new_df[columns_to_normalize] = min_max_scaler.fit_transform(new_df[columns_to_normalize])
    #print(new_df.zip_code.unique())
    return new_df

def clean_data(old_df):
    df = old_df.copy()
    columns_to_drop = ['id', 'member_id', 'emp_title']
    df = df.drop(columns_to_drop, axis = 1)
    
    cols = ['zip_code', 'funded_amnt', 'emp_length', 'home_ownership', 
            'int_rate', 'purpose', 'total_pymnt','annual_inc', 
            'verification_status', 'dti', 'loan_status', 'revol_util', 
            'grade', 'term']
    
    cols_encode = ['emp_length', 'home_ownership', 'verification_status', 'grade', 'purpose']
    cols_normalize = ['funded_amnt', 'annual_inc', 'dti']
    
    df = initial_model(df, cols, cols_encode, cols_normalize)
    
    df['term'] = df['term'].map(format_term, na_action = 'ignore')
    df['int_rate'] = df['int_rate'].map(format_interest, na_action='ignore')
    df['revol_util'] = df['revol_util'].map(format_revol, na_action='ignore')
    
    df['zip_code'] = df['zip_code'].str.slice(0,3)
    
    df = df.dropna()
    df = df[df.loan_status != 'Current']
    
    df['paid'] = 1;
    df.loc[df.loan_status == 'Fully Paid', 'paid'] = 0;
    
    return df

In [111]:
df_2014 = clean_data(data_2014)

/Users/hugoramambason/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [113]:
df_2014.shape

(218391, 49)

In [114]:
df_2015 = clean_data(data_2015)

/Users/hugoramambason/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [115]:
df_2015 = df_2015.drop(['purpose_educational'], axis = 1)

In [116]:
df_2015.columns == df_2014.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [119]:
df_2014 = df_2014.dropna()
df_2015 = df_2015.dropna()

In [124]:
df_2014.to_csv('cleaned_2014', index = False)
df_2015.to_csv('cleaned_2015', index = False)